In [1]:
from DbConnector import DbConnector
from part2 import DbExecutor

In [2]:
# Set up the program
program = DbConnector()
executor = DbExecutor()

Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('assignment2',)
-----------------------------------------------

Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('assignment2',)
-----------------------------------------------



### Task 1

How many users, activities and trackpoints are there in the dataset (after it is inserted into the database)

In [3]:
# Get the number of Users
users = executor.get_table_size("User")
print(f"There are {users[0]} users")

# Get the number of Activities
activities = executor.get_table_size("Activity")
print(f"There are {activities[0]} activities")

# Get the number of TrackPoints
trackpoints = executor.get_table_size("TrackPoint")
print(f"There are {trackpoints[0]} trackpoints")


There are (182,) users
There are (21057,) activities
There are (5355109,) trackpoints


### Task 2a

In [4]:
# Get the average number of trackpoints per user
users = executor.findAvgTrackpoints()

users = "".join([f'\n user: {user[0]} \t trackpoints: {user[1]}' for user in users])
print(f"Average number of trackpoins per user: {users}")


Average number of trackpoins per user: 
 user: 000 	 trackpoints: 670.8710
 user: 001 	 trackpoints: 824.9649
 user: 002 	 trackpoints: 924.8014
 user: 003 	 trackpoints: 807.3870
 user: 004 	 trackpoints: 761.8584
 user: 005 	 trackpoints: 587.5479
 user: 006 	 trackpoints: 801.1250
 user: 007 	 trackpoints: 758.2750
 user: 008 	 trackpoints: 1376.0000
 user: 009 	 trackpoints: 1218.3243
 user: 010 	 trackpoints: 1107.5000
 user: 011 	 trackpoints: 451.7562
 user: 012 	 trackpoints: 629.7826
 user: 013 	 trackpoints: 1388.3025
 user: 014 	 trackpoints: 905.9364
 user: 015 	 trackpoints: 806.5833
 user: 016 	 trackpoints: 690.2222
 user: 017 	 trackpoints: 868.2453
 user: 018 	 trackpoints: 540.7955
 user: 019 	 trackpoints: 372.2911
 user: 020 	 trackpoints: 704.6383
 user: 021 	 trackpoints: 475.0000
 user: 022 	 trackpoints: 900.6951
 user: 023 	 trackpoints: 979.3125
 user: 024 	 trackpoints: 860.4082
 user: 025 	 trackpoints: 606.2951
 user: 026 	 trackpoints: 1034.7143
 user: 027

In [5]:
# Get the minimum number of trackpoints per user
users = executor.findMinTrackpoints()

users = "".join([f'\n user: {user[0]} \t trackpoints: {user[1]}' for user in users])

print(f"Minimum number of trackpoins per user: {users}")

Minimum number of trackpoins per user: 
 user: 000 	 trackpoints: 5
 user: 001 	 trackpoints: 33
 user: 002 	 trackpoints: 4
 user: 003 	 trackpoints: 3
 user: 004 	 trackpoints: 4
 user: 005 	 trackpoints: 5
 user: 006 	 trackpoints: 14
 user: 007 	 trackpoints: 6
 user: 008 	 trackpoints: 165
 user: 009 	 trackpoints: 134
 user: 010 	 trackpoints: 663
 user: 011 	 trackpoints: 3
 user: 012 	 trackpoints: 64
 user: 013 	 trackpoints: 13
 user: 014 	 trackpoints: 3
 user: 015 	 trackpoints: 22
 user: 016 	 trackpoints: 7
 user: 017 	 trackpoints: 4
 user: 018 	 trackpoints: 3
 user: 019 	 trackpoints: 11
 user: 020 	 trackpoints: 22
 user: 021 	 trackpoints: 475
 user: 022 	 trackpoints: 15
 user: 023 	 trackpoints: 31
 user: 024 	 trackpoints: 4
 user: 025 	 trackpoints: 3
 user: 026 	 trackpoints: 9
 user: 027 	 trackpoints: 1760
 user: 028 	 trackpoints: 32
 user: 029 	 trackpoints: 4
 user: 030 	 trackpoints: 9
 user: 031 	 trackpoints: 23
 user: 032 	 trackpoints: 62
 user: 033 	 

In [6]:
# Get the maximum number of trackpoints per user
users = executor.findMaxTrackpoints()
users = "".join([f'\n user: {user[0]} \t transportations: {user[1]}' for user in users])
print(f"Maximum number of trackpoins per user: {users}")

Maximum number of trackpoins per user: 
 user: 000 	 transportations: 2359
 user: 001 	 transportations: 2472
 user: 002 	 transportations: 2438
 user: 003 	 transportations: 2485
 user: 004 	 transportations: 2482
 user: 005 	 transportations: 2058
 user: 006 	 transportations: 2478
 user: 007 	 transportations: 2228
 user: 008 	 transportations: 2499
 user: 009 	 transportations: 2396
 user: 010 	 transportations: 1964
 user: 011 	 transportations: 2306
 user: 012 	 transportations: 2277
 user: 013 	 transportations: 2486
 user: 014 	 transportations: 2499
 user: 015 	 transportations: 2411
 user: 016 	 transportations: 2360
 user: 017 	 transportations: 2471
 user: 018 	 transportations: 2245
 user: 019 	 transportations: 2276
 user: 020 	 transportations: 2201
 user: 021 	 transportations: 475
 user: 022 	 transportations: 2421
 user: 023 	 transportations: 2215
 user: 024 	 transportations: 2377
 user: 025 	 transportations: 2464
 user: 026 	 transportations: 2396
 user: 027 	 tra

### Task 4

Find all users who have taken a bus.

In [7]:
# Get the users who have taken the bus
users = executor.get_user_taken_bus()
users = "".join([f'\n user: {user[0]}' for user in users[0]])
print(f"The user ids for user that have taken the bus: {users}")

The user ids for user that have taken the bus: 
 user: 010
 user: 020
 user: 052
 user: 053
 user: 058
 user: 062
 user: 064
 user: 065
 user: 067
 user: 068
 user: 069
 user: 073
 user: 075
 user: 078
 user: 080
 user: 081
 user: 082
 user: 084
 user: 085
 user: 088
 user: 091
 user: 092
 user: 096
 user: 098
 user: 100
 user: 101
 user: 102
 user: 104
 user: 105
 user: 108
 user: 110
 user: 111
 user: 112
 user: 114
 user: 124
 user: 125
 user: 126
 user: 128
 user: 129
 user: 138
 user: 139
 user: 141
 user: 147
 user: 153
 user: 154
 user: 161
 user: 163
 user: 167
 user: 174
 user: 175
 user: 179


### Task 5

In [8]:
users, _ = executor.findTop10TransportationsUsers()

users = "".join([f'\n user: {user[0]} \t transportations: {user[1]}' for user in users])
print(f"Top 10 users with the most unique transportation modes: {users}")

Top 10 users with the most unique transportation modes: 
 user: 128 	 transportations: 10
 user: 075 	 transportations: 9
 user: 062 	 transportations: 9
 user: 163 	 transportations: 9
 user: 084 	 transportations: 8
 user: 153 	 transportations: 8
 user: 126 	 transportations: 8
 user: 167 	 transportations: 8
 user: 085 	 transportations: 7
 user: 010 	 transportations: 7


### Task 7

a) Find the number of users that have started an activity in one day and ended the activity the next day.

In [9]:
users = executor.get_user_activity_over_a_day()

num = set()
for user in users[0]:
    num.add(user[0])

count = len(num)

print(f"Number of users with activity that ends the next day: {count}")

Number of users with activity that ends the next day: 95


b) List the transportation mode, user id and duration for these activities.

In [10]:
users = "".join([f'\n user: {user[0]} transportation mode: {user[1]} duration: {user[2]}' for user in users[0]])
print(f"The user who have an end date one day after the start day: {users}")
print(users[0])

The user who have an end date one day after the start day: 
 user: 000 transportation mode: None duration: 4:09:15
 user: 000 transportation mode: None duration: 0:10:45
 user: 000 transportation mode: None duration: 11:03:55
 user: 000 transportation mode: None duration: 1:43:20
 user: 000 transportation mode: None duration: 16:24:06
 user: 001 transportation mode: None duration: 6:54:46
 user: 001 transportation mode: None duration: 0:09:30
 user: 001 transportation mode: None duration: 0:24:57
 user: 001 transportation mode: None duration: 14:17:28
 user: 001 transportation mode: None duration: 8:26:34
 user: 001 transportation mode: None duration: 1:19:41
 user: 001 transportation mode: None duration: 0:55:48
 user: 001 transportation mode: None duration: 0:58:08
 user: 001 transportation mode: None duration: 0:42:03
 user: 001 transportation mode: None duration: 13:30:14
 user: 001 transportation mode: None duration: 2:40:20
 user: 001 transportation mode: None duration: 0:46:26
 

### Task 8

In [14]:
users, _ = executor.findCloseUsers()

users

OperationalError: 2013 (HY000): Lost connection to MySQL server during query

### Task 10

Find the users that have traveled the longest total distance in one day for each
transportation mode.

### Task 11

In [12]:
users, _ = executor.findInvalidActivities()

users = "".join([f'\n user: {user[0]} \t invalid activites: {user[1]}' for user in users])
print(f"The users that have invalid activities and the number of invalid activities each users has: {users}")

The users that have invalid activities and the number of invalid activities each users has: 
 user: 000 	 invalid activites: 442
 user: 001 	 invalid activites: 116
 user: 002 	 invalid activites: 295
 user: 003 	 invalid activites: 839
 user: 004 	 invalid activites: 1169
 user: 005 	 invalid activites: 126
 user: 006 	 invalid activites: 48
 user: 007 	 invalid activites: 133
 user: 008 	 invalid activites: 33
 user: 009 	 invalid activites: 114
 user: 011 	 invalid activites: 38
 user: 012 	 invalid activites: 163
 user: 013 	 invalid activites: 68
 user: 014 	 invalid activites: 328
 user: 015 	 invalid activites: 136
 user: 016 	 invalid activites: 60
 user: 017 	 invalid activites: 492
 user: 018 	 invalid activites: 97
 user: 019 	 invalid activites: 65
 user: 020 	 invalid activites: 15
 user: 021 	 invalid activites: 1
 user: 022 	 invalid activites: 188
 user: 023 	 invalid activites: 30
 user: 024 	 invalid activites: 73
 user: 025 	 invalid activites: 424
 user: 026 	 inval

In [13]:
# Closing the connection
program.close_connection()


-----------------------------------------------
Connection to 8.0.34-0ubuntu0.22.04.1 is closed
